In [1]:
import numpy as np
import numpy.random as rgt
from scipy.stats import norm, t
import time
import pandas as pd

from conquer.linear import low_dim
rgt.seed(2023)

# number of monte carlo simulations
M = 500 

# Homoscedastic model

In [2]:
n, p = 8000, 400
itcp, beta = 4, 1*np.ones(p)*(2*rgt.binomial(1, 1/2, p) - 1)
tau, t_df = 0.75, 2
runtime = 0

itcp_se, coef_se = np.empty(M), np.empty(M)
for m in range(M):
    X = rgt.normal(0, 1.5, size=(n,p))
    Y = itcp + X.dot(beta) + rgt.standard_t(t_df, n) - t.ppf(tau, t_df)

    tic = time.time()
    model = low_dim(X, Y).fit(tau=tau)
    runtime += time.time() - tic

    itcp_se[m] = (model['beta'][0] - itcp)**2
    coef_se[m] = np.sum((model['beta'][1:] - beta)**2)

In [3]:
out = {'MSE (itcp)': np.mean(itcp_se), 
       'std (itcp)': np.std(itcp_se), 
       'MSE (coef)': np.mean(coef_se),
       'std (coef)': np.std(coef_se),
       'Runtime': runtime/M}
out = pd.DataFrame(out, index=['conquer'])
out

,MSE (itcp),std (itcp),MSE (coef),std (coef),Runtime
conquer,0.001783,0.001698,0.076375,0.00594,0.049663


### Construction of confidence intervals

In [4]:
n, p = 500, 20
mask = 2*rgt.binomial(1, 1/2, p) - 1
itcp, beta = 4, 1*np.ones(p)*mask
tau, t_df = 0.75, 2

ci_cover = np.zeros([4, p])
ci_width = np.empty([M, 4, p])
for m in range(M):
    X = rgt.normal(0, 1.5, size=(n,p))
    Y = itcp + X.dot(beta) + rgt.standard_t(t_df, n) - t.ppf(tau, t_df)

    sqr = low_dim(X, Y)    
    model1 = sqr.norm_ci(tau)
    model2 = sqr.mb_ci(tau)
    
    ci_cover[0,:] += (beta >= model1['normal_ci'][1:,0])*(beta<= model1['normal_ci'][1:,1])
    ci_cover[1,:] += (beta >= model2['percentile_ci'][1:,0])*(beta<= model2['percentile_ci'][1:,1])
    ci_cover[2,:] += (beta >= model2['pivotal_ci'][1:,0])*(beta<= model2['pivotal_ci'][1:,1])
    ci_cover[3,:] += (beta >= model2['normal_ci'][1:,0])*(beta<= model2['normal_ci'][1:,1])
    
    ci_width[m,0,:] = model1['normal_ci'][1:,1] - model1['normal_ci'][1:,0]
    ci_width[m,1,:] = model2['percentile_ci'][1:,1] - model2['percentile_ci'][1:,0]
    ci_width[m,2,:] = model2['pivotal_ci'][1:,1] - model2['pivotal_ci'][1:,0]
    ci_width[m,3,:] = model2['normal_ci'][1:,1] - model2['normal_ci'][1:,0]

In [5]:
cover = pd.DataFrame(ci_cover/M, index=["Normal", "MB-Percentile", "MB-Pivotal", "MB-Normal"])
cover.columns = pd.Index(np.linspace(1,p,p), dtype=int)
cover

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
Normal,0.954,0.956,0.964,0.968,0.960,0.968,0.974,0.948,0.944,0.958,0.948,0.968,0.956,0.970,0.974,0.966,0.968,0.958,0.956,0.976
MB-Percentile,0.956,0.964,0.964,0.962,0.960,0.964,0.980,0.952,0.950,0.954,0.950,0.970,0.966,0.964,0.978,0.966,0.958,0.956,0.956,0.974
MB-Pivotal,0.926,0.916,0.910,0.936,0.926,0.922,0.944,0.914,0.926,0.920,0.922,0.928,0.916,0.924,0.922,0.932,0.912,0.920,0.926,0.946
MB-Normal,0.956,0.960,0.956,0.966,0.954,0.962,0.972,0.948,0.950,0.948,0.944,0.962,0.954,0.954,0.962,0.960,0.960,0.956,0.950,0.966


In [6]:
width = pd.DataFrame(np.mean(ci_width, axis=0), index=["Normal", "MB-Percentile", "MB-Pivotal", "MB-Normal"])
width.columns = cover.columns
width

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
Normal,0.258636,0.258380,0.257166,0.255176,0.254863,0.258789,0.258385,0.256334,0.259101,0.260862,0.259542,0.260677,0.260635,0.258821,0.258130,0.260719,0.259682,0.260531,0.257781,0.258585
MB-Percentile,0.223395,0.225000,0.224694,0.223747,0.222957,0.222936,0.223719,0.221991,0.222199,0.225471,0.224225,0.225290,0.224470,0.224572,0.223628,0.223647,0.224542,0.224745,0.222705,0.223346
MB-Pivotal,0.223395,0.225000,0.224694,0.223747,0.222957,0.222936,0.223719,0.221991,0.222199,0.225471,0.224225,0.225290,0.224470,0.224572,0.223628,0.223647,0.224542,0.224745,0.222705,0.223346
MB-Normal,0.226959,0.228427,0.228302,0.226479,0.226361,0.226278,0.226970,0.225772,0.226569,0.228913,0.227835,0.229003,0.228084,0.227950,0.226883,0.228274,0.228908,0.228356,0.226696,0.227392


# Heteroscedastic model

Let $z=(z_1, \ldots, z_p)^T \sim N(0, \Sigma)$ with $\Sigma = (0.5^{|j-k|})_{1\leq j, k \leq p}$ and $z_0 \sim {\rm Unif}(0,2)$ be independent. Generate independent data vectors $\{(y_i , x_i) \}_{i=1}^n$ from the model 
$$
    y_i =  \varepsilon_i x_{i1}  +  x_{i2} + \cdots + x_{ip}   \quad {\rm with } \ \  x_i = (x_{i1}, \ldots, x_{ip})^T \sim (z_0, z_2, \ldots, z_p)^T,
$$
where $\varepsilon_i$'s are iid $N(0,1)$ variables that are independent of $x_i$'s.

Consider two quantile levels: $\tau=0.5$ and $\tau=0.8$. Note that the effect of $x_{i1}$ is only present for $\tau=0.8$.

In [7]:
def cov_generate(std, corr=0.5):
    p = len(std)
    R = np.zeros(shape=[p,p])
    for j in range(p-1):
        R[j, j+1:] = np.array(range(1, len(R[j,j+1:])+1))
    R += R.T
    return np.outer(std, std) * (corr*np.ones(shape=[p,p]))** R
        
n = 2000
p = 10
mu, Sig = np.zeros(p), cov_generate(np.ones(p), 0.5)
beta = np.ones(p)
beta[0] = 0

### Case 1: $\tau=0.5$.
The conditional median of $y_i$ given $x_i$ is $Q_{0.5}(y_i | x_i) =  x_{i2} + \cdots + x_{ip}$.

In [8]:
tau = 0.5
ci_cover = np.zeros([4, p])
ci_width = np.empty([M, 4, p])
for m in range(M):
    X = rgt.multivariate_normal(mean=mu, cov=Sig, size=n)
    X[:,0] = rgt.uniform(0, 2, size=n)
    Y = X.dot(beta) +  X[:,0]*rgt.normal(0,1,size=n)

    qr = low_dim(X, Y, intercept=False)    
    model1 = qr.norm_ci(tau)
    model2 = qr.mb_ci(tau)
    
    ci_cover[0,:] += (beta >= model1['normal_ci'][:,0])*(beta<= model1['normal_ci'][:,1])
    ci_cover[1,:] += (beta >= model2['percentile_ci'][:,0])*(beta<= model2['percentile_ci'][:,1])
    ci_cover[2,:] += (beta >= model2['pivotal_ci'][:,0])*(beta<= model2['pivotal_ci'][:,1])
    ci_cover[3,:] += (beta >= model2['normal_ci'][:,0])*(beta<= model2['normal_ci'][:,1])
    
    ci_width[m,0,:] = model1['normal_ci'][:,1] - model1['normal_ci'][:,0]
    ci_width[m,1,:] = model2['percentile_ci'][:,1] - model2['percentile_ci'][:,0]
    ci_width[m,2,:] = model2['pivotal_ci'][:,1] - model2['pivotal_ci'][:,0]
    ci_width[m,3,:] = model2['normal_ci'][:,1] - model2['normal_ci'][:,0]

In [9]:
cover = pd.DataFrame(ci_cover/M, index=["Normal", "MB-Percentile", "MB-Pivotal", "MB-Normal"])
cover.columns = pd.Index(np.linspace(1,p,p), dtype=int)
cover

,1,2,3,4,5,6,7,8,9,10
Normal,0.932,0.964,0.962,0.968,0.938,0.954,0.954,0.954,0.958,0.942
MB-Percentile,0.928,0.954,0.950,0.958,0.928,0.942,0.948,0.942,0.952,0.932
MB-Pivotal,0.920,0.962,0.972,0.976,0.942,0.956,0.956,0.962,0.968,0.958
MB-Normal,0.926,0.962,0.964,0.974,0.946,0.962,0.962,0.960,0.966,0.946


In [10]:
width = pd.DataFrame(np.mean(ci_width, axis=0), index=["Normal", "MB-Percentile", "MB-Pivotal", "MB-Normal"])
width.columns = cover.columns
width

,1,2,3,4,5,6,7,8,9,10
Normal,0.125196,0.062774,0.070305,0.070226,0.069601,0.070413,0.070121,0.070036,0.069831,0.062752
MB-Percentile,0.121011,0.064799,0.072251,0.072284,0.071941,0.072065,0.071832,0.072270,0.072317,0.064818
MB-Pivotal,0.121011,0.064799,0.072251,0.072284,0.071941,0.072065,0.071832,0.072270,0.072317,0.064818
MB-Normal,0.123750,0.065728,0.073649,0.073518,0.073099,0.073555,0.073175,0.073457,0.073430,0.065833


### Case 2: $\tau=0.8$. 
In this case, the conditional $0.8$-quantile of $y_i$ given $x_i$ is $Q_{0.8}(y_i | x_i) =   \Phi^{-1}(0.8) x_{i1} + x_{i2} + \cdots + x_{ip}$.

In [11]:
tau = 0.8
true_beta = np.copy(beta)
true_beta[0] = norm.ppf(tau)

ci_cover = np.zeros([4, p])
ci_width = np.empty([M, 4, p])
for m in range(M):
    X = rgt.multivariate_normal(mean=mu, cov=Sig, size=n)
    X[:,0] = rgt.uniform(0, 2, size=n)
    Y = X.dot(beta) + X[:,0]*rgt.normal(0,1,size=n)

    qr = low_dim(X, Y, intercept=False)    
    model1 = qr.norm_ci(tau)
    model2 = qr.mb_ci(tau)
    
    ci_cover[0,:] += (true_beta>=model1['normal_ci'][:,0])*(true_beta<= model1['normal_ci'][:,1])
    ci_cover[1,:] += (true_beta>=model2['percentile_ci'][:,0])*(true_beta<= model2['percentile_ci'][:,1])
    ci_cover[2,:] += (true_beta>=model2['pivotal_ci'][:,0])*(true_beta<= model2['pivotal_ci'][:,1])
    ci_cover[3,:] += (true_beta>=model2['normal_ci'][:,0])*(true_beta<= model2['normal_ci'][:,1])
    
    ci_width[m,0,:] = model1['normal_ci'][:,1] - model1['normal_ci'][:,0]
    ci_width[m,1,:] = model2['percentile_ci'][:,1] - model2['percentile_ci'][:,0]
    ci_width[m,2,:] = model2['pivotal_ci'][:,1] - model2['pivotal_ci'][:,0]
    ci_width[m,3,:] = model2['normal_ci'][:,1] - model2['normal_ci'][:,0]
        
cover = pd.DataFrame(ci_cover/M, index=["Normal", "MB-Percentile", "MB-Pivotal", "MB-Normal"])
cover.columns = pd.Index(np.linspace(1,p,p), dtype=int)

width = pd.DataFrame(np.mean(ci_width, axis=0), index=["Normal", "MB-Percentile", "MB-Pivotal", "MB-Normal"])
width.columns = cover.columns

In [12]:
cover

,1,2,3,4,5,6,7,8,9,10
Normal,0.966,0.948,0.950,0.962,0.944,0.948,0.958,0.956,0.954,0.958
MB-Percentile,0.962,0.938,0.944,0.952,0.934,0.938,0.948,0.944,0.950,0.954
MB-Pivotal,0.948,0.956,0.958,0.968,0.966,0.960,0.974,0.960,0.962,0.974
MB-Normal,0.966,0.958,0.956,0.966,0.952,0.956,0.968,0.962,0.964,0.974


In [13]:
width

,1,2,3,4,5,6,7,8,9,10
Normal,0.141892,0.064906,0.072184,0.072694,0.072338,0.072651,0.072825,0.072701,0.073100,0.064731
MB-Percentile,0.137464,0.066781,0.075030,0.075114,0.075231,0.075796,0.075338,0.075579,0.075391,0.067308
MB-Pivotal,0.137464,0.066781,0.075030,0.075114,0.075231,0.075796,0.075338,0.075579,0.075391,0.067308
MB-Normal,0.140429,0.068264,0.076140,0.076252,0.076290,0.076738,0.076724,0.076526,0.076737,0.068255
